## Red wine quality classifier using a Neural network

### Perform EDA, Explore the features using histograms, any data preprocessing required:::

### NN using Tensorflow Estimator DNN Classifier implementation:::
### Neural Net using Keras implementation:::
### Training & Validation Accuracy plots::::
### Training & Validation Loss plots:::
### Conclusion:::